In [1]:
## Imports
import os
import sys
import time
import json


## Data importation

In [2]:
## Paths
dirname = os.getcwd()
rel_dataset_export_path = 'OneDrive - Association Cesi Viacesi mail/Bureau/Projects/winAi/Datasets/processed/stats-bomb'
dataset_export_path = os.path.join(dirname, rel_dataset_export_path)

In [3]:
## Load the most recent dataset
dataset_files = os.listdir(dataset_export_path)
dataset_files.sort()
dataset_file = dataset_files[-1]
dataset_path = os.path.join(dataset_export_path, dataset_file)

print(f"Loading dataset from {dataset_path}")

Loading dataset from C:\Users\NyveK\OneDrive - Association Cesi Viacesi mail/Bureau/Projects/winAi/Datasets/processed/stats-bomb\20240805-1441_matches_stats.csv
